# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import pickle
import nltk
import spacy
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
%matplotlib inline
#nltk.download('punkt')
import tensorflow_hub as hub
import tensorflow as tf


# Se leen los .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.loc[train['id'].isin(ids_with_target_error),'target'] = 0

In [5]:
test = pd.read_csv('csv/test.csv')

In [6]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# Se leen las abreviaturas 
file = open('abreviaturas.pkl','rb')
abbreviations = pickle.load(file)
file.close()

# Elaboración del modelo

Para este modelo se aplicará NLP para el procesamiento de los tweets y se utilizarán distintos árboles de decisión para realizar las predicciones:

### Preparación del set de datos

In [8]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

In [9]:
# Devuelve lista de tweets con abreviaturas expandidas
def expand_abbreviations(sentences):
    expanded_sentences = []
    
    for sentence in sentences:
        expanded_sentence = []
        
        for word in sentence:
            if word in abbreviations:
                word = word.replace(word,abbreviations[word])
            expanded_sentence.append(word)
            
        tokenized = nltk.word_tokenize(" ".join(expanded_sentence))
        expanded_sentences.append(tokenized)
        
    return expanded_sentences

In [10]:
#Devuelve los tweets en minuscula eliminando simbolos y numeros
def remove_symbols_and_numbers(sentences):
    clean_sentences = []

    for sentence in sentences:
        clean_sentence = []
    
        for word in sentence:
            clean_word = re.sub('[^a-zA-Z]',' ', word)
            if(clean_word != ' '):
                clean_sentence.append(clean_word.lower())
                
        tokenized = nltk.word_tokenize(" ".join(clean_sentence))
        clean_sentences.append(tokenized)

    return clean_sentences

In [11]:
def remove_stopwords(sentences):
    clean_sentences = []
    for sentence in sentences:
        clean_sentence = []
        for word in sentence:
            if word not in STOPWORDS:
                clean_sentence.append(word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

In [12]:
# PARA QUE CORRA ESTA FUNCION HAY QUE PONER EN CONSOLA:
# pip install spacy
# spacy download en
# Devuelve los tweets lematizados
def lemmatize_tweets(sentences):
    nlp = spacy.load('en')
    lemmatized = []
    
    for sentence in sentences:
        sentence = ' '.join(sentence)
        doc = nlp(sentence)
        lemmatized.append(' '.join([token.lemma_ for token in doc]) )
    
    return lemmatized    

In [13]:
#Devuelve una lista de tweets dejando solo letras y lematizando las palabras
def clean_text(df):
    
    words = df['text'].str.split()
    words = remove_symbols_and_numbers(words)
    words = expand_abbreviations(words)
    words = remove_stopwords(words)
    
    return lemmatize_tweets(words)

In [14]:
train['clean_text'] = clean_text(train)
train.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake allah forgive
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo ruby alaska smoke wildfire pour...


In [15]:
test['clean_text'] = clean_text(test)
test.head()

,id,keyword,location,text,clean_text
0,0,NaN,NaN,Just happened a terrible car crash,happen terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",hear earthquake different city stay safe
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest spot pond geese flee street save
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


### Embedding de los tweets con ELMo

In [16]:
tf.compat.v1.disable_eager_execution()
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [17]:
def elmo_vectoring(tweets):
    
    embeddings = elmo(tweets.tolist(), signature="default", as_dict=True)["elmo"]
    
    tf_session = tf.compat.v1.Session()
    tf_session.run(tf.compat.v1.global_variables_initializer())
    tf_session.run(tf.compat.v1.tables_initializer())
    
    return tf_session.run(tf.reduce_mean(embeddings,1))

In [18]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [19]:
elmo_train = [elmo_vectoring(x['clean_text']) for x in list_train]
elmo_test = [elmo_vectoring(x['clean_text']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [20]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [21]:
train_embedding_file = open("train_embedding_file.txt", "w")

for i in elmo_train_new:
    np.savetxt(train_embedding_file, i)

train_embedding_file.close()

In [22]:
test_embedding_file = open("test_embedding_file.txt", "w")

for i in elmo_test_new:
    np.savetxt(test_embedding_file, i)

test_embedding_file.close()

In [23]:
elmo_train = np.loadtxt("train_embedding_file.txt").reshape(len(train.index),1024)
elmo_train.shape

(7613, 1024)

In [24]:
elmo_test = np.loadtxt("test_embedding_file.txt").reshape(len(test.index),1024)
elmo_test.shape

(3263, 1024)

### Se entrena el modelo

In [27]:
X, y = elmo_train, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [29]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 5)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.7716535433070866
Precision Score : 0.8007662835249042
Recall Score : 0.6314199395770392
F1 Score : 0.706081081081081


In [30]:
rf_model = RandomForestRegressor(random_state=13, n_estimators=5, max_depth=10)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.7388451443569554
Precision Score : 0.7444444444444445
Recall Score : 0.6072507552870091
F1 Score : 0.6688851913477538


In [31]:
lgb_class = lgb.LGBMRegressor(learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 5)
lgb_class.fit(X_train, y_train)
preds = lgb_class.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.7545931758530183
Precision Score : 0.8461538461538461
Recall Score : 0.5317220543806647
F1 Score : 0.6530612244897959


In [32]:
catb = CatBoostRegressor(iterations=5, depth=10)
catb.fit(X_train, y_train)
preds = catb.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Learning rate set to 0.5
0:	learn: 0.4380765	total: 1.03s	remaining: 4.12s
1:	learn: 0.4013788	total: 2.02s	remaining: 3.03s
2:	learn: 0.3768230	total: 2.96s	remaining: 1.97s
3:	learn: 0.3630068	total: 3.93s	remaining: 983ms
4:	learn: 0.3470293	total: 4.85s	remaining: 0us
Accuracy Score : 0.7454068241469817
Precision Score : 0.7665369649805448
Recall Score : 0.595166163141994
F1 Score : 0.6700680272108844


In [33]:
gb = GradientBoostingRegressor(n_estimators=5, learning_rate=0.1, 
                                max_features=2, max_depth = 10, random_state = 0)
gb.fit(X_train, y_train)
preds = gb.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.7230971128608924
Precision Score : 0.7912621359223301
Recall Score : 0.49244712990936557
F1 Score : 0.6070763500931098


In [34]:
eclf2 = VotingRegressor(estimators=[
         ('xgb', xg_reg), ('rf', rf_model), ('catb', catb), ('gb', gb), ('lgbm',lgb_class)])

eclf2 = eclf2.fit(X, y)
preds = eclf2.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Learning rate set to 0.5
0:	learn: 0.4372127	total: 925ms	remaining: 3.7s
1:	learn: 0.3996522	total: 1.86s	remaining: 2.79s
2:	learn: 0.3783464	total: 2.81s	remaining: 1.87s
3:	learn: 0.3587327	total: 3.75s	remaining: 937ms
4:	learn: 0.3457851	total: 4.67s	remaining: 0us
Accuracy Score : 0.910761154855643
Precision Score : 0.9646643109540636
Recall Score : 0.824773413897281
F1 Score : 0.8892508143322475


In [35]:
df = pd.DataFrame(eclf2.predict(X),columns=["elmo_score"])
df.head()

,elmo_score
0,0.548083
1,0.601624
2,0.643861
3,0.529898
4,0.559565


In [36]:
final = df.to_csv('csv/solo_embedding_elmo_train.csv')

### Predicciones

In [37]:
test['target'] = eclf2.predict(elmo_test)

In [38]:
test.drop(columns=['clean_text'], inplace=True)
test.set_index('id', inplace=True)
test.head(10)

,keyword,location,text,target
id,,,,
0,NaN,NaN,Just happened a terrible car crash,0.269342
2,NaN,NaN,"Heard about #earthquake is different cities, s...",0.294223
3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0.489406
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0.326129
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0.317996
12,NaN,NaN,We're shaking...It's an earthquake,0.254549
21,NaN,NaN,They'd probably still show more life than Arse...,0.263589
22,NaN,NaN,Hey! How are you?,0.259348
27,NaN,NaN,What a nice hat?,0.254998


In [39]:
final = test.to_csv('csv/submission_elmo.csv')

In [40]:
df = pd.DataFrame(eclf2.predict(elmo_test),columns=["elmo_score"])
df.head()
final = df.to_csv('csv/solo_embedding_elmo_test.csv')